In the next sections we will work through examples of using the KerasClassifier wrapper for a classification neural network created in Keras and used in the scikit-learn library. 

9.2 Evaluate Deep Learning Models with Cross Valida-tion

The KerasClassifier and KerasRegressor classes in Keras take an argument build_fn which is the name of the function to call to create your model.

In the example below we define a function create model() that create a simple multi-layer neural network for the problem.

We pass this function name to the KerasClassifier class by the build fn argument.

In [1]:
# MLP for Pima Indians Dataset with 10-fold cross validation via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.cross_validation import StratifiedKFold
from sklearn.model_selection import cross_val_score
import numpy
import pandas

Using TensorFlow backend.
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# Function to create model, required for KerasClassifier
def create_model():
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, init='uniform', activation='relu')) 
    model.add(Dense(8, init='uniform', activation='relu')) 
    model.add(Dense(1, init='uniform', activation='sigmoid'))
  # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) 
    return model

In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [4]:
# create model
model = KerasClassifier(build_fn=create_model, nb_epoch=150, batch_size=10)
# evaluate using 10-fold cross validation
kfold = StratifiedKFold(y=Y, n_folds=10, shuffle=True, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=8, activation="relu", kernel_initializer="uniform")`
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`


Epoch 1/1
77/77 [==============================] - 0s 716us/step
Epoch 1/1
77/77 [==============================] - 0s 844us/step
Epoch 1/1
77/77 [==============================] - 0s 1ms/step
Epoch 1/1
77/77 [==============================] - 0s 1ms/step
Epoch 1/1
77/77 [==============================] - 0s 2ms/step
Epoch 1/1
77/77 [==============================] - 0s 2ms/step
Epoch 1/1
77/77 [==============================] - 0s 2ms/step
Epoch 1/1
77/77 [==============================] - 0s 2ms/step
Epoch 1/1
76/76 [==============================] - 0s 3ms/step
Epoch 1/1
76/76 [==============================] - 0s 3ms/step
0.6510594735488611


### 9.3 Grid Search Deep Learning Model Parameters

In this example we use a grid search to evaluate different configurations for our neural network model and report on the combination that provides the best estimated performance.

The create model() function is defined to take two arguments optimizer and init, both of which must have default values. 

- Optimizers for searching different weight values.
- Initializers for preparing the network weights using different schemes.
- Number of epochs for training the model for different number of exposures to the training dataset.
- Batches for varying the number of samples before weight updates.

In [5]:
# MLP for Pima Indians Dataset with grid search via sklearn
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.grid_search import GridSearchCV
import numpy
import pandas

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [6]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
# create model
    model = Sequential()
    model.add(Dense(12, input_dim=8, init=init, activation='relu')) 
    model.add(Dense(8, init=init, activation='relu')) 
    model.add(Dense(1, init=init, activation='sigmoid'))
  # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
    return model

In [7]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [8]:
# create model
model = KerasClassifier(build_fn=create_model)
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = numpy.array([50, 100, 150])
batches = numpy.array([5, 10, 20])

In [9]:
param_grid = dict(optimizer=optimizers, nb_epoch=epochs, batch_size=batches, init=init) 
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [10]:
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
for params, mean_score, scores in grid_result.grid_scores_:
    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=8, activation="relu", kernel_initializer="glorot_uniform")`
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="glorot_uniform")`
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="glorot_uniform")`


Epoch 1/1
256/256 [==============================] - 0s 1ms/step
Epoch 1/1
256/256 [==============================] - 0s 1ms/step
Epoch 1/1
256/256 [==============================] - 0s 1ms/step
Epoch 1/1
256/256 [==============================] - 0s 1ms/step
Epoch 1/1
256/256 [==============================] - 0s 2ms/step
Epoch 1/1
256/256 [==============================] - 0s 2ms/step
Epoch 1/1
256/256 [==============================] - 0s 2ms/step
Epoch 1/1
256/256 [==============================] - 0s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 2ms/step
Epoch 1/1
256/256 [==============================] - 1s 3ms/step
Epoch 1/1
256/256 [==============================] - 1s 3ms/step
Epoch 1/1
256/256 [==============================] - 1s 3ms/step
Epoch 1/1
256/256 [==============================] - 1s 3ms/step
Epoch 1/1
256/256 [======

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=8, activation="relu", kernel_initializer="normal")`
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="normal")`
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="normal")`


Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 4ms/step
Epoch 1/1
256/256 [==============================] - 1s 5ms/step
Epoch 1/1
256/256 [==============================] - 1s 5ms/step
Epoch 1/1
256/256 [==============================] - 1s 5ms/step
Epoch 1/1
256/256 [==============================] - 1s 5ms/step
Epoch 1/1
256/256 [==============================] - 1s 5ms/step
Epoch 1/1
256/256 [==============================] - 2s 6ms/step
Epoch 1/1
256/256 [==============================] - 1s 6ms/step
Epoch 1/1
256/256 [==============================] - 2s 7ms/step
Epoch 1/1
256/256 [==============================] - 2s 6ms/step
Epoch 1/1
256/256 [==============================] - 2s 6ms/step
Epoch 1/1
256/256 [==============================] - 2s 6ms/step
Epoch 1/1
256/256 [======

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(12, input_dim=8, activation="relu", kernel_initializer="uniform")`
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(8, activation="relu", kernel_initializer="uniform")`
/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`


Epoch 1/1
256/256 [==============================] - 2s 7ms/step
Epoch 1/1
256/256 [==============================] - 2s 7ms/step
Epoch 1/1
256/256 [==============================] - 2s 8ms/step
Epoch 1/1
256/256 [==============================] - 2s 8ms/step
Epoch 1/1
256/256 [==============================] - 2s 8ms/step
Epoch 1/1
256/256 [==============================] - 2s 8ms/step
Epoch 1/1
256/256 [==============================] - 2s 9ms/step
Epoch 1/1
256/256 [==============================] - 2s 8ms/step
Epoch 1/1
256/256 [==============================] - 2s 9ms/step
Epoch 1/1
256/256 [==============================] - 2s 9ms/step
Epoch 1/1
256/256 [==============================] - 2s 10ms/step
Epoch 1/1
256/256 [==============================] - 2s 9ms/step
Epoch 1/1
256/256 [==============================] - 3s 10ms/step
Epoch 1/1
256/256 [==============================] - 3s 10ms/step
Epoch 1/1
256/256 [==============================] - 3s 10ms/step
Epoch 1/1
256/256 [==

256/256 [==============================] - 8s 32ms/step
Epoch 1/1
256/256 [==============================] - 8s 32ms/step
Epoch 1/1
256/256 [==============================] - 9s 33ms/step
Epoch 1/1
256/256 [==============================] - 9s 33ms/step
Epoch 1/1
256/256 [==============================] - 9s 34ms/step
Epoch 1/1
256/256 [==============================] - 9s 34ms/step
Epoch 1/1
256/256 [==============================] - 9s 36ms/step
Epoch 1/1
256/256 [==============================] - 9s 34ms/step
Epoch 1/1
256/256 [==============================] - 9s 37ms/step
Epoch 1/1
256/256 [==============================] - 9s 34ms/step
Epoch 1/1
256/256 [==============================] - 9s 37ms/step
Epoch 1/1
256/256 [==============================] - 9s 35ms/step
Epoch 1/1
256/256 [==============================] - 9s 36ms/step
Epoch 1/1
256/256 [==============================] - 9s 36ms/step
Epoch 1/1
256/256 [==============================] - 9s 37ms/step
Epoch 1/1
256/256 [=